# Install neccessary libraries

In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime

# pre processing (obsolete now)

In [1]:
%matplotlib inline

In [ ]:
# Make folder of images and labels from one combined folder of images and labels
import os
import shutil
import glob
import random
from PIL import Image
import cv2
import matplotlib.pyplot as plt
# Create folder of images and labels from one combined folder of images and labels
input_path = "/home/harsh/FYP/poly/polypd"
img_path = "/home/harsh/FYP/poly/images"
label_path = "/home/harsh/FYP/poly/labels"

# images and labes have the same name
# get all images of format .jp and tif convert all tif images to jpg
# for file in glob.glob(os.path.join(input_path, "*.tif")):
#     print(file)
#     im = cv2.imread(file)
#     im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
#     cv2.imwrite(file[:-4]+".jpg", im)

# get all images of format .jpg
count = 0
for file in glob.glob(os.path.join(input_path, "*.jpg")):
    print(file)
    count += 1
    # create folder for images
    shutil.copy(file, img_path)
    # create folder for labels
    shutil.copy(file[:-4]+".txt", label_path)

# YOLO training part

In [2]:
from ultralytics import YOLO

In [ ]:
#Use YOLOv8 
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="/home/harsh/FYP/poly/poly.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
path = model.export(format="onnx")  # export the model to ONNX format

# Code to compile video from the frames

In [ ]:
# the given path contains all the folders of frames
folder_path = "/home/harsh/FYP/poly/masks"
#each frame folder has a image folder and mask folder
# image folder contains the image of that frame
# mask folder contains the mask of that frame
# Compile video from the frames
import cv2
import os
import glob
import re
import numpy as np
from PIL import Image

# get all the folders of frames
folders = glob.glob(os.path.join(folder_path, "*"))
# sort the folders according to the frame number
folders.sort(key=lambda f: int(re.sub('\D', '', f)))
print(folders)
img_list = []
# iterate through all the frame folders to get the images
for folder in folders:
    # get all the images of that frame
    images = glob.glob(os.path.join(folder, "images", "*.jpg"))
    # sort the images according to the image number
    images.sort(key=lambda f: int(re.sub('\D', '', f)))
    img_list.append(images)
    # print(images)
    # get the first image of the frame
    # run the below line in loop only after the first iteration
print(img_list)
img = cv2.imread(images[0])
# get the height and width of the image
height, width, layers = img.shape
# create a video writer object
video = cv2.VideoWriter("poly"+".avi", 0, 30, (width,height))
# iterate through all the images of the frame
for image in img_list:
    # read the image
    img = cv2.imread(image[0])
    # write the image to the video
    video.write(img)
# release the video writer object
video.release()



# DeepSORT

In [3]:
# using the DeepSORT tracker with yolov8
import cv2
import numpy as np
import sys
import glob
import time
import torch
import torchvision.transforms as transforms
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from deep_sort_realtime.deep_sort.detection import Detection
from collections import Counter

transform = transforms.Compose([transforms.ToTensor()])

# Load YOLOv5 model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("best.pt")  # load a pretrained model (recommended for training)
classes = ['polyp']
my_tracker = DeepSort(max_age=5,max_cosine_distance=0.8, nms_max_overlap=1.0)

# Open video file
cap = cv2.VideoCapture("poly.avi")
# Define the codec and create VideoWriter object if you want to save the video

out = cv2.VideoWriter("eval.avi", 0, 30, (1920,1080))
# Define colors for different tracks
color_list = np.random.randint(0, 255, size=(1000, 3), dtype=np.uint8)
# Loop through frames
while True:
    # Read frame
    ret, frame = cap.read()
    start = time.perf_counter()
    if not ret:
        break
    # print(frame.shape)
    # Detect objects with YOLOv8
    results = model(frame)
    # print(results[0].boxes)
    # Extract features for each object
    deepsort_detections = []
    features = []
    for obj in results[0].boxes.data:
        x1, y1, x2, y2, conf, cls = obj.cpu().numpy()
        if conf > 0.3 and cls==0:
            # #for any pretrained model uncomment this if you have a any
            # cropped = frame[int(y1):int(y2), int(x1):int(x2)]
            # resized = cv2.resize(cropped, (224, 224))
            # tensor = torch.from_numpy(resized).permute(2, 0, 1).float().div(255.0).unsqueeze(0).cuda()
            # with torch.no_grad():
            #     feature = model.forward(tensor)
            #     feature = feature.squeeze().cpu().numpy()
            # feature = feature[:,-1].squeeze()
            # features.append(feature)
            # print(feature.shape)
            detection = Detection(np.array([x1, y1, int(x2-x1), int(y2-y1)]), conf, feature = None,class_name='polypd')
            deepsort_detections.append((list(detection.get_ltwh()), detection.confidence, detection.class_name))

    # Associate objects across frames with DeepSORT
    ids = []
    total = 0
    if len(deepsort_detections) > 0:
        # print("here",len(detections))
        # Run DeepSORT tracker
        deepsort_tracks = my_tracker.update_tracks(raw_detections = deepsort_detections, frame = frame)
        # Draw bounding boxes and track IDs on frame
        for track in deepsort_tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            ltrb = track.to_ltrb()
            ids.append(int(track_id))
            bbox = ltrb
            
            cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),(color_list[int(track_id)%1000].tolist()),2)
            cv2.putText(frame, "ID: " + str(track_id), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (color_list[int(track_id)%1000].tolist()), 2)
    # Display output frame
        total = len(Counter(ids).keys())
    end = time.perf_counter()
    totaltime = end - start
    fps = 1 / totaltime
    cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    cv2.putText(frame, f'Count: {int(total)}', (20,20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
    # cv2.imshow('frame', frame)
    # cv2.show()
    out.write(frame)
    # break  
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
# cap.release()
out.release()
cv2.destroyAllWindows()


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128